In [ ]:
# To convert from NFA to DFA using subset construction method
# 1. Epsilon closure of the initial state
# 2. For each state in the epsilon closure, find the transition on each input symbol
# 3. Epsilon closure of the states reached in step 2
# 4. Repeat step 2 and 3 until no new states are reached
# 5. The states reached in step 4 are the states of the DFA
# 6. The transition function of the DFA is the transitions found in step 2 and 3

In [1]:
Symbols = ['a', 'b', 'c']
accept_states = {3}
start_state = {0}

In [7]:
def epsilon_closure(nfa, states):
    closure = set(states)
    stack = list(states)
    while stack:
        state = stack.pop()
        if 'e' in nfa[state]:
            for s in nfa[state]['e']:
                if s not in closure:
                    closure.add(s)
                    stack.append(s)
    return closure

def move(nfa, states, symbol):
    moves = set()
    for state in states:
        if symbol in nfa[state]:
            moves.update(nfa[state][symbol])
    return moves

nfa = {
    0: {'a': {3} , 'b': {1}},
    1: {'a': {0}, 'b': {2}},
    2: {'b': {1} , 'a' :{3}},
    3: {'a' : {2} , 'b' : {0}},
}
D_states = epsilon_closure(nfa,start_state) # {0, 1}
print(([D_states]))
D_transitions = {}
def convert_dfa_from_nfa(nfa , D_states , D_transitions) :
    stack = [D_states]
    while stack:
        D_states = stack.pop()
        for symbol in Symbols:
            moves = move(nfa, D_states, symbol)
            moves = epsilon_closure(nfa, moves)
            if moves:
                if (frozenset(D_states), symbol) not in D_transitions:
                    D_transitions[(frozenset(D_states), symbol)] = moves
                    stack.append(moves)
    return D_transitions


D_transitions = convert_dfa_from_nfa(nfa , D_states , D_transitions)

D_states = set(D_transitions.keys())
set_of_states = set()
dict_of_states = {}
for D_state in D_states:
    set_of_states.add(D_state[0])
for state in (set_of_states):
    for accept_state in accept_states:
        if accept_state in state:
            dict_of_states[str(set(state))] = True
            break
        else:
            dict_of_states[str(set(state))] = False
print(dict_of_states)


[{0}]
{'{3}': True, '{2}': False, '{1}': False, '{0}': False}


In [3]:
from graphviz import Digraph
def visualize_dfa(D_transitions):
    dot = Digraph()

    # Add nodes
    for key in dict_of_states.keys():
        dot.node(key, shape='doublecircle' if dict_of_states[key] else 'circle')

    # Add edges
    for (src, symbol), dst in D_transitions.items():
        dot.edge(str(set(src)), str(set(dst)), label=symbol)

    return dot


#D_transitions = convert_dfa_from_nfa(nfa , D_states , D_transitions)
dot = visualize_dfa(D_transitions)
dot.format = 'png'
dot.render('dfa_graph')

'dfa_graph.png'

In [106]:
print(D_transitions)
# D_transitions = {(''.join(str(e) for e in key[0]), str(key[1])): value for key, value in D_transitions.items()}
# D_transitions = {(''.join(str(e) for e in key[0]), str(key[1])): str(value) for key, value in D_transitions.items()}

{(frozenset({0}), 'a'): {3}, (frozenset({0}), 'b'): {1}, (frozenset({1}), 'a'): {0}, (frozenset({1}), 'b'): {2}, (frozenset({2}), 'a'): {3}, (frozenset({2}), 'b'): {1}, (frozenset({3}), 'a'): {2}, (frozenset({3}), 'b'): {0}}


{('frozenset({0})', 'a'): {3}, ('frozenset({0})', 'b'): {1}, ('frozenset({1})', 'a'): {0}, ('frozenset({1})', 'b'): {2}, ('frozenset({2})', 'a'): {3}, ('frozenset({2})', 'b'): {1}, ('frozenset({3})', 'a'): {2}, ('frozenset({3})', 'b'): {0}}


In [22]:
new_D_transitions = {}

# Iterate through the original dictionary
for key, value in D_transitions.items():
    # Extract state and symbol from the key
    state = ', '.join(str(s) for s in key[0])
    symbol = key[1]
    # Construct new key in the desired format
    new_key = f'{{{state}}}', symbol
    new_D_transitions[new_key] = value

print(new_D_transitions)



{('{0}', 'a'): {3}, ('{0}', 'b'): {1}, ('{1}', 'a'): {0}, ('{1}', 'b'): {2}, ('{2}', 'a'): {3}, ('{2}', 'b'): {1}, ('{3}', 'a'): {2}, ('{3}', 'b'): {0}}


In [55]:
print(dict_of_states)

{'{3}': True, '{2}': False, '{1}': False, '{0}': False}


In [69]:
first_set = set()
second_set = set()
for key in dict_of_states.keys():
    if dict_of_states[key]:
        second_set.add(key)
    else:
        first_set.add(key)
stack = [first_set, second_set]
def hopcroft(D_transitions, Symbols, dict_of_states):
    P = [first_set, second_set]  # Initial partition
    W = [second_set]  # New states to be examined
    
    while W:
        A = W.pop()
        for c in Symbols:
            X = set()
            for state in dict_of_states.keys():
                if (state, c) in D_transitions:
                    if str(D_transitions[(state, c)]) in A:
                        X.add(state)
            for Y in P[:]:
                intersection = X & Y
                difference = Y - X
                if intersection:
                    P.remove(Y)
                    P.extend([intersection, difference])
                    if Y in W:
                        W.remove(Y)
                        W.extend([intersection, difference])
                    else:
                        if len(intersection) <= len(difference):
                            W.append(intersection)
                        else:
                            W.append(difference)
    if set() in P:
        P.remove(set())  
    return P

P = hopcroft(new_D_transitions, Symbols, dict_of_states)
print(P)


[{'{3}'}, {'{1}'}, {'{0}', '{2}'}]


{2}
{3}
{3}
{3}
yes
{0}
{3}
{3}
{3}
yes
